In [1]:
%cd ..

/home/jo3/p/features2image_diffusion


/home/jo3/p/features2image_diffusion/.venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import argtoml
import jax
import matplotlib.pyplot as plt
import numpy as np
import torch

from pathlib import Path
from typing import Tuple

from jaxtyping import Float
from jo3mnist.vis import to_img
from torch.utils.data import DataLoader

from features2image_diffusion.__main__ import evaluate
from features2image_diffusion.data import load_mnist_with_features, loader_to_dataframe
from features2image_diffusion.unet import load_ddpm

np.set_printoptions(precision=3)

RUN_PATH = Path.cwd() / "run/7927ad448cfdb96d1e819f599d5ce8a200a3de67068866289c8cf1e0215ec398"
O = argtoml.parse_args(toml_path=RUN_PATH / "config.toml")
RUN = O.run[0]
DEVICE = "cuda"

print("create the dataloader")
TRAIN_LOADER, TEST_LOADER = load_mnist_with_features(
    RUN.feature_dir, O.mnist_dir, RUN.batch_size, shuffle=False
)
print("convert the loader to a dataframe")
TRAIN_SET = loader_to_dataframe(TRAIN_LOADER, img_dir="./res/mnist/img")
TRAIN_SET

create the dataloader


/home/jo3/p/features2image_diffusion/.venv/lib/python3.11/site-packages/jo3util/warning.py:18: ToDoWarning: 'Make sure file names are numeric before the .npy.'
  warnings.warn(msg, ToDoWarning)


convert the loader to a dataframe
[1/2] loading labels


938it [00:15, 60.57it/s]


[2/2] loading features


100%|██████████| 256/256 [00:00<00:00, 105186.31it/s]


,label,f0,f1,f2,f3,f4,f5,f6,f7,f8,...,f246,f247,f248,f249,f250,f251,f252,f253,f254,f255
0,5,0.0,0.000000,0.0,1.161055,0.0,3.392065,1.578063,0.0,0.864978,...,0.000000,1.805129,0.690105,0.0,0.0,5.241269,3.842837,0.176452,1.565828,2.654130
1,0,0.0,0.000000,0.0,0.000000,0.0,2.358853,1.813100,0.0,0.979746,...,0.000000,1.363689,0.324502,0.0,0.0,4.079500,4.786779,0.565612,1.361328,1.053584
2,4,0.0,0.671618,0.0,2.503571,0.0,2.140285,2.176373,0.0,0.000000,...,0.798268,1.046827,5.871449,0.0,0.0,3.325204,0.699517,1.559370,0.000000,0.000000
3,1,0.0,1.423953,0.0,1.501160,0.0,3.335502,0.000000,0.0,0.000000,...,0.966482,0.770367,3.258740,0.0,0.0,0.427500,0.728990,1.242967,0.000000,1.957382
4,9,0.0,1.553129,0.0,1.257297,0.0,2.169177,2.704326,0.0,0.000000,...,1.234310,1.416535,4.576626,0.0,0.0,4.055480,1.079787,2.771267,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,8,0.0,1.231961,0.0,1.562882,0.0,0.851112,1.040367,0.0,0.000000,...,0.567958,0.203369,2.788358,0.0,0.0,0.652251,3.938322,2.780792,1.568819,1.649010
59996,3,0.0,0.000000,0.0,1.200623,0.0,1.093581,0.922486,0.0,0.000000,...,0.000000,0.889216,2.278658,0.0,0.0,2.478583,3.761220,1.109611,1.830709,2.852156
59997,5,0.0,0.000000,0.0,1.269054,0.0,3.882545,2.461601,0.0,0.414531,...,0.000000,1.437299,0.571311,0.0,0.0,4.221604,3.603690,1.651947,1.986321,0.207435
59998,6,0.0,0.000000,0.0,2.222346,0.0,3.886921,1.917095,0.0,0.000000,...,0.000000,1.065447,2.913041,0.0,0.0,3.674189,2.821726,1.130351,1.015811,1.842106
